# Data Simulation

### Features

- date
- demand_google searches
- is_holiday 
- day_of_week
- is_weekend
- is_holiday
- offer_type
- social_spend
- sem_spend
- other_spend
- brand_traffic
- gross_sales


### Features description

- date
- demand_google searches
- is_holiday 
- day_of_week
- is_weekend
- is_holiday
- offer_type
- social_spend
- sem_spend
- other_spend
- brand_traffic
- gross_sales